# RAG查询优化：多查询与查询转换技术

在基础RAG系统中，我们使用单一查询进行检索。但在实际应用中，用户的查询往往存在表达模糊、角度单一或过于笼统的问题。本章将介绍多种查询优化技术，让你的RAG系统能够更准确地理解用户意图。

## 查询优化的必要性

### 单一查询的局限性

```python

用户查询: "机器学习是什么？"

# 可能错过的相关文档：
- "深度学习入门" (使用了不同但相关的术语)
- "AI算法基础" (更广泛的主题)
- "神经网络原理" (具体技术)
- "监督学习vs无监督学习" (细分话题)
```

### 查询优化如何帮助？

查询优化的核心思想：通过生成多个角度的查询、重写查询或分解复杂查询，增加检索到相关文档的概率。

```python
# ✅ 查询优化后

原始查询: "机器学习是什么？"

生成的变体:
1. "什么是机器学习算法？"
2. "机器学习的基本概念和原理"
3. "AI中的机器学习技术"
4. "机器学习的应用场景"

→ 检索 → 合并结果 → 去重 → 生成答案
```

## 技术概览

本章将介绍5种主要的查询优化技术：

| 技术 | 核心思想 | 适用场景 | 复杂度 |
|:-----|:---------|:---------|:-------|
| Multi-Query | 生成查询的多个变体 | 用户查询表达不清 | ⭐ |
| RAG-Fusion | 多查询+重排序融合 | 需要高质量结果 | ⭐⭐ |
| Decomposition | 分解复杂查询 | 多步骤问题 | ⭐⭐⭐ |
| Step Back | 先问概括性问题 | 需要背景知识 | ⭐⭐ |
| HyDE | 生成假设性文档 | 语义搜索增强 | ⭐⭐⭐ |

In [2]:
# 模型准备工作
import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from vllm import LLM

# 1. 加载本地嵌入模型
local_model_path = "./Models/maidalun/bce-embedding-base_v1" 
embeddings = HuggingFaceEmbeddings(
    model_name=local_model_path,
    model_kwargs={"device": "cuda"},
    encode_kwargs={"normalize_embeddings": True}
)

# 2. 加载本地向量数据库
vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )

# 3. 加载本地大模型
model_dir="../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8"
os.environ['VLLM_USE_MODELSCOPE'] = 'True'
llm = LLM(
    model=model_dir,
    tokenizer=model_dir,
    trust_remote_code=True
)

/root/shared-nvme/conda-envs/py310/lib/python3.10/site-packages/_distutils_hack/__init__.py:53: UserWarning: Reliance on distutils from stdlib is deprecated. Users must rely on setuptools to provide the distutils module. Avoid importing distutils or import setuptools first, and avoid setting SETUPTOOLS_USE_DISTUTILS=stdlib. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


INFO 11-11 09:52:04 config.py:177] The model is convertible to Marlin format. Using Marlin kernel.
INFO 11-11 09:52:04 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8', speculative_config=None, tokenizer='../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=8192, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=gptq_marlin, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=../Qwen-vllm/Models/Qwen/Qwen-7B-Chat-Int8)
WARNING 11-11 09:52:05 tokenizer.py:126] Using a slow tokenizer. This might cause a significant slowdown. Consider using a fast tokenizer instead.
INFO 11-11 09:52:05 u

2025-11-11 09:52:15,933 - modelscope - INFO - PyTorch version 2.3.0+cu118 Found.
2025-11-11 09:52:15,935 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2025-11-11 09:52:16,000 - modelscope - INFO - Loading done! Current index file version is 1.12.0, with md5 298ceecce207285dd10b135af16e71cc and a total number of 964 components indexed


INFO 11-11 09:52:29 model_runner.py:175] Loading model weights took 8.4983 GB
INFO 11-11 09:52:32 gpu_executor.py:114] # GPU blocks: 1404, # CPU blocks: 512
INFO 11-11 09:52:36 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 11-11 09:52:36 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 11-11 09:52:46 model_runner.py:1017] Graph capturing finished in 9 secs.


## Part 1：Multi-Query - 多角度查询

核心思想：Multi-Query技术通过LLM生成原始查询的多个变体，从不同角度检索文档，提高检索的召回率。

工作流程：

```python
用户查询: "什么是Agent？"
    ↓
使用LLM生成变体:
    ├─ "Agent系统的定义是什么？"
    ├─ "AI Agent的核心概念"
    └─ "什么是自主智能体？"
    ↓
并行检索每个变体
    ↓
合并并去重结果
    ↓
生成最终答案
```

In [13]:
import os
import chromadb
from vllm import LLM, SamplingParams
from modelscope import snapshot_download
from langchain_community.embeddings import HuggingFaceEmbeddings
from typing import List, Dict, Any

def get_unique_documents(documents: List[List]) -> List:
    """去重文档"""
    unique_docs = {}
    for doc_list in documents:
        for doc in doc_list:
            content = doc.page_content
            if content not in unique_docs:
                unique_docs[content] = doc
    return list(unique_docs.values())

class MultiQueryRAG:
    """完全使用ChatML格式的Multi-Query RAG系统"""
    
    def __init__(self, vectorstore, llm):
        """
        初始化Multi-Query RAG
        
        Args:
            vectorstore: 向量数据库
            llm: 语言模型
        """
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
        self.llm = llm
        print("✅ Multi-Query RAG系统初始化完成")
    
    def _generate_query_variants(self, question: str) -> List[str]:
        """使用ChatML格式生成查询变体"""
        query_prompt = f"""<|im_start|>system
            你是一个AI助手，擅长将用户的问题改写成多个语义相同但表达不同的搜索查询。每个查询应简洁、独立，适合用于向量检索。<|im_end|>
            <|im_start|>user
            原始问题：{question}
            
            请生成3个不同的搜索查询，每行一个，不要编号，不要解释：<|im_end|>
            <|im_start|>assistant
            """
        
        sampling_params = SamplingParams(
            max_tokens=500,
            temperature=0.7,
            top_p=0.9,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([query_prompt], sampling_params)
        response = outputs[0].outputs[0].text.strip()
        
        # 解析生成的查询
        queries = []
        lines = response.split('\n')
        for line in lines:
            line = line.strip()
            if line and len(line) > 5:
                # 移除可能的编号标记
                if line[0].isdigit() and (line[1] == '.' or line[1] == '、'):
                    query = line[2:].strip()
                else:
                    query = line
                queries.append(query)
        
        # 确保包含原始问题并去重
        all_queries = [question] + queries
        unique_queries = []
        seen = set()
        for q in all_queries:
            if q not in seen:
                seen.add(q)
                unique_queries.append(q)
        
        return unique_queries[:4]  # 最多返回4个查询
    
    def _retrieve_documents(self, queries: List[str]) -> List:
        """检索所有查询的文档"""
        all_docs = []
        for query in queries:
            try:
                docs = self.retriever.get_relevant_documents(query)
                all_docs.append(docs)
                print(f"   🔍 '{query}': 找到 {len(docs)} 个文档")
            except Exception as e:
                print(f"❌ 检索失败 '{query}': {e}")
                all_docs.append([])
        return all_docs
    
    def _process_documents(self, all_docs: List[List]) -> List:
        """处理并去重文档"""
        unique_docs = get_unique_documents(all_docs)
        return unique_docs[:10]  # 返回top-10
    
    def _clean_context(self, text: str) -> str:
        """清理上下文内容"""
        import re
        if not text or not isinstance(text, str):
            return text
        
        # 清理格式
        cleaned = re.sub(r'(?<=[^\s])\n(?=[^\s])', '', text)  # 合并错误分割的文字
        cleaned = re.sub(r'\n\s+\n', '\n\n', cleaned)  # 压缩空行
        cleaned = re.sub(r'\n\s+', '\n', cleaned)  # 清理行内空格
        cleaned = re.sub(r'[ \t]{2,}', ' ', cleaned)  # 清理连续空格
        cleaned = cleaned.strip()  # 清理首尾空格
        
        return cleaned
    
    def _build_answer_prompt(self, question: str, context: str) -> str:
        """构建ChatML格式的答案生成提示词"""
        cleaned_context = self._clean_context(context)
        
        return f"""<|im_start|>system
            你是一个专业的AI助手。请严格基于以下上下文信息回答问题：
            
            {cleaned_context}
            
            请遵循以下规则：
            1. 只使用上下文中的信息回答
            2. 如果上下文不包含相关信息，请回答"我不知道"
            3. 保持回答准确、简洁
            4. 不要编造信息<|im_end|>
            <|im_start|>user
            {question}<|im_end|>
            <|im_start|>assistant
            """
    
    def _generate_answer(self, prompt: str, max_tokens: int = 512, temperature: float = 0.3) -> str:
        """生成答案"""
        sampling_params = SamplingParams(
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        return outputs[0].outputs[0].text.strip()
    
    def query(self, question: str, max_tokens: int = 512, temperature: float = 0.3) -> Dict[str, Any]:
        """执行Multi-Query RAG查询"""
        print(f"🎯 原始问题: {question}")
        print("-" * 50)
        
        # Step 1: 生成查询变体
        queries = self._generate_query_variants(question)
        print(f"📝 生成了 {len(queries)} 个查询:")
        for i, q in enumerate(queries, 1):
            print(f"   {i}. {q}")
        
        # Step 2: 检索文档
        all_docs = self._retrieve_documents(queries)
        total_docs_before_dedup = sum(len(docs) for docs in all_docs)
        print(f"📚 检索到 {total_docs_before_dedup} 个文档（含重复）")
        
        # Step 3: 去重
        unique_docs = self._process_documents(all_docs)
        print(f"✨ 去重后剩余 {len(unique_docs)} 个文档")
        
        # Step 4: 构建上下文
        context = "\n\n".join([doc.page_content for doc in unique_docs])
        
        # Step 5: 生成答案
        prompt = self._build_answer_prompt(question, context)
        answer = self._generate_answer(prompt, max_tokens, temperature)
        
        # 返回结果
        return {
            "question": question,
            "answer": answer,
            "queries": queries,
            "num_docs": len(unique_docs),
            "total_docs_before_dedup": total_docs_before_dedup,
            "context_preview": context[:200] + "..." if len(context) > 200 else context
        }
    
    def batch_query(self, questions: List[str], **kwargs) -> List[Dict]:
        """批量查询"""
        results = []
        for i, question in enumerate(questions, 1):
            print(f"\n🔍 处理第 {i}/{len(questions)} 个问题...")
            result = self.query(question, **kwargs)
            results.append(result)
        return results



# === 使用示例 ===
if __name__ == "__main__":
    print("🚀 ChatML格式的Multi-Query RAG")
    print("=" * 60)
    
    # 1. 创建实例
    multi_rag = MultiQueryRAG(vectorstore, llm)
    
    # 2. 执行查询
    result = multi_rag.query("什么是机械臂？它有什么能力？")
    
    # 3. 查看结果
    print("\n" + "="*60)
    print("💡 答案:", result["answer"])
    print("📝 使用的查询:", result["queries"])
    print("📚 参考文档数:", result["num_docs"])


🚀 ChatML格式的Multi-Query RAG
✅ Multi-Query RAG系统初始化完成
🎯 原始问题: 什么是机械臂？它有什么能力？
--------------------------------------------------


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  2.55it/s]


📝 生成了 4 个查询:
   1. 什么是机械臂？它有什么能力？
   2. 机械臂是什么？
   3. 机械臂有哪些功能？
   4. 机械臂的工作原理是什么？
   🔍 '什么是机械臂？它有什么能力？': 找到 3 个文档
   🔍 '机械臂是什么？': 找到 3 个文档
   🔍 '机械臂有哪些功能？': 找到 3 个文档
   🔍 '机械臂的工作原理是什么？': 找到 3 个文档
📚 检索到 12 个文档（含重复）
✨ 去重后剩余 6 个文档


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.24it/s]


💡 答案: 机械臂是一种能够自动执行任务的机器人手臂，它由多个关节组成，可以按照预设的程序或指令进行精确的运动。机械臂具有强大的力量和精确度，可以完成各种复杂的任务，例如抓取、搬运、装配等。
📝 使用的查询: ['什么是机械臂？它有什么能力？', '机械臂是什么？', '机械臂有哪些功能？', '机械臂的工作原理是什么？']
📚 参考文档数: 6


Multi-Query的优缺点:

优点：提高召回率（找到更多相关文档）；覆盖不同角度和表达方式；对用户表达不清的查询特别有效；实现相对简单

缺点：增加检索成本（多次查询）；可能引入噪声（不相关的变体）；需要额外的LLM调用；去重逻辑可能过滤掉有价值的文档

优化技巧：

1. 限制查询数量

```python
return unique_queries[:4]  # 最多返回4个查询
```

2. 使用缓存避免重复生成

```python
from functools import lru_cache
@lru_cache(maxsize=100)
def cached_query_generation(question: str) -> tuple:
    queries = query_generator.invoke({"question": question})
    return tuple(queries)  # 返回元组以支持缓存
```

3. 异步并行检索

```python
import asyncio
async def async_retrieve_all(queries: List[str], retriever):
    """异步并行检索"""
    tasks = [retriever.aget_relevant_documents(q) for q in queries]
    results = await asyncio.gather(*tasks)
    return results
```

## Part 2: RAG-Fusion - 融合式检索

核心概念：RAG-Fusion结合了Multi-Query和倒数排序融合（Reciprocal Rank Fusion, RRF），不仅生成多个查询，还智能地合并和重排序结果。

什么是倒数排序融合？

RRF是一种排序融合算法，给予排名靠前的文档更高的分数：

```python
公式: RRF_score(doc) = Σ 1 / (k + rank(doc))

其中:
- k = 常数（通常为60）
- rank(doc) = 文档在某个查询结果中的排名
- Σ = 对所有查询结果求和
```

示例：

```python
# 查询1的结果排序:
查询1: "什么是Agent?"
  1. 文档A (排名1)
  2. 文档B (排名2)
  3. 文档C (排名3)

# 查询2的结果排序:
查询2: "AI Agent的定义"
  1. 文档B (排名1)  ← 在这个查询中排名更高
  2. 文档D (排名2)
  3. 文档A (排名3)

# RRF融合计算 (k=60):
文档A: 1/(60+1) + 1/(60+3) = 0.0164 + 0.0159 = 0.0323
文档B: 1/(60+2) + 1/(60+1) = 0.0161 + 0.0164 = 0.0325 ← 最高分
文档C: 1/(60+3) + 0         = 0.0159
文档D: 0         + 1/(60+2) = 0.0161

最终排序: B > A > D > C
```

In [4]:
from typing import List, Tuple
from vllm import SamplingParams

def reciprocal_rank_fusion(
    results: List[List],
    k: int = 60
) -> List[Tuple[any, float]]:
    """
    实现倒数排序融合
    
    Args:
        results: 多个查询的结果列表
        k: RRF常数
    
    Returns:
        排序后的(文档, 分数)列表
    """
    # 存储每个文档的融合分数
    fusion_scores = {}
    
    for docs in results:
        for rank, doc in enumerate(docs):
            # 使用文档内容作为唯一标识
            doc_id = doc.page_content
            
            # 计算RRF分数
            if doc_id not in fusion_scores:
                fusion_scores[doc_id] = {
                    'doc': doc,
                    'score': 0
                }
            
            # 累加分数
            fusion_scores[doc_id]['score'] += 1 / (k + rank + 1)
    
    # 按分数排序
    sorted_docs = sorted(
        fusion_scores.values(),
        key=lambda x: x['score'],
        reverse=True
    )
    
    return [(item['doc'], item['score']) for item in sorted_docs]


class RAGFusion:
    """使用ChatML格式的RAG-Fusion系统"""
    
    def __init__(self, vectorstore, llm, k: int = 60):
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
        self.llm = llm
        self.k = k
        
    def generate_queries(self, question: str, n: int = 3) -> List[str]:
        """使用ChatML格式生成查询变体"""
        prompt = f"""<|im_start|>system
            你是一个AI助手，擅长将用户的问题改写成多个语义相同但表达不同的搜索查询。
            请生成{n}个不同角度的查询变体。
            
            要求：
            1. 语义相关但表达不同
            2. 覆盖不同角度
            3. 每行一个查询<|im_end|>
            <|im_start|>user
            原始问题：{question}
            
            请生成{n}个不同的搜索查询，每行一个：<|im_end|>
            <|im_start|>assistant
            """
        
        sampling_params = SamplingParams(
            max_tokens=512,
            temperature=0.7,
            top_p=0.9,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        response = outputs[0].outputs[0].text.strip()
        
        # 解析生成的查询
        queries = []
        lines = response.split('\n')
        for line in lines:
            line = line.strip()
            if line and len(line) > 5:  # 过滤太短的查询
                # 移除可能的编号标记
                if line[0].isdigit() and (line[1] == '.' or line[1] == '、'):
                    query = line[2:].strip()
                else:
                    query = line
                queries.append(query)
        
        # 确保包含原始问题并去重
        all_queries = [question] + queries
        unique_queries = []
        seen = set()
        for q in all_queries:
            if q not in seen:
                seen.add(q)
                unique_queries.append(q)
        
        return unique_queries[:n+1]  # 包含原始问题
    
    def retrieve_and_fuse(self, queries: List[str]) -> List[Tuple[any, float]]:
        """检索并融合结果"""
        all_results = []
        
        print(f"🔍 执行 {len(queries)} 个查询...")
        for i, query in enumerate(queries, 1):
            docs = self.retriever.get_relevant_documents(query)
            all_results.append(docs)
            print(f"   查询{i}: '{query}' -> 检索到 {len(docs)} 个文档")
        
        # RRF融合
        fused_results = reciprocal_rank_fusion(all_results, k=self.k)
        print(f"✨ 融合后共 {len(fused_results)} 个唯一文档")
        
        return fused_results
    
    def _build_answer_prompt(self, question: str, context: str) -> str:
        """构建ChatML格式的答案生成提示词"""
        return f"""<|im_start|>system
            你是一个专业的AI助手。请基于以下按相关性排序的文档回答问题。
            
            {context}
            
            请遵循以下规则：
            1. 只使用文档中的信息回答
            2. 如果文档不包含相关信息，请回答"我不知道"
            3. 保持回答准确、简洁<|im_end|>
            <|im_start|>user
            {question}<|im_end|>
            <|im_start|>assistant
            """
    
    def _generate_answer(self, prompt: str, max_tokens: int = 512, temperature: float = 0.3) -> str:
        """生成答案"""
        sampling_params = SamplingParams(
            max_tokens=max_tokens,
            temperature=temperature,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        return outputs[0].outputs[0].text.strip()
    
    def query(self, question: str, top_k: int = 5, max_tokens: int = 512, temperature: float = 0.3) -> dict:
        """执行RAG-Fusion查询"""
        print(f"🎯 原始问题: {question}")
        print("-" * 50)
        
        # 1. 生成查询
        queries = self.generate_queries(question)
        print(f"📝 生成了 {len(queries)} 个查询:")
        for i, q in enumerate(queries, 1):
            print(f"   {i}. {q}")
        
        # 2. 检索并融合
        fused_docs = self.retrieve_and_fuse(queries)
        
        # 3. 选择top-k
        top_docs = fused_docs[:top_k]
        print(f"🏆 选择Top-{len(top_docs)} 文档:")
        for i, (doc, score) in enumerate(top_docs, 1):
            preview = doc.page_content[:100] + "..." if len(doc.page_content) > 100 else doc.page_content
            print(f"   {i}. [分数: {score:.4f}] {preview}")
        
        # 4. 生成答案
        context = "\n\n".join([
            f"[文档{i+1} 分数: {score:.4f}]\n{doc.page_content}" 
            for i, (doc, score) in enumerate(top_docs)
        ])
        
        prompt = self._build_answer_prompt(question, context)
        answer = self._generate_answer(prompt, max_tokens, temperature)
        
        return {
            "question": question,
            "generated_queries": queries,
            "num_docs": len(top_docs),
            "top_scores": [score for _, score in top_docs],
            "answer": answer,
            "context_preview": context[:200] + "..." if len(context) > 200 else context
        }
    
    def batch_query(self, questions: List[str], **kwargs) -> List[dict]:
        """批量查询"""
        results = []
        for i, question in enumerate(questions, 1):
            print(f"\n🔍 处理第 {i}/{len(questions)} 个问题...")
            result = self.query(question, **kwargs)
            results.append(result)
        return results

In [6]:
print("🚀 RAG-Fusion系统")
print("=" * 60)

# 1. 创建实例
rag_fusion = RAGFusion(vectorstore, llm, k=60)

# 2. 执行查询
result = rag_fusion.query("什么是机械臂？它有哪些关键能力？")

# 3. 查看结果
print("\n" + "="*60)
print("生成的查询:")
for q in result["generated_queries"]:
    print(f"  - {q}")

print(f"\nTop-{result['num_docs']} 文档分数:")
for i, score in enumerate(result["top_scores"], 1):
    print(f"  {i}. {score:.4f}")

print("\n答案:")
print(result["answer"])

🚀 RAG-Fusion系统
🎯 原始问题: 什么是机械臂？它有哪些关键能力？
--------------------------------------------------


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.57it/s]


📝 生成了 4 个查询:
   1. 什么是机械臂？它有哪些关键能力？
   2. 机械臂是什么？它的主要功能是什么？
   3. 机械臂有哪些重要的特点和功能？
   4. 机械臂的运作原理是什么？它有哪些关键能力？
🔍 执行 4 个查询...
   查询1: '什么是机械臂？它有哪些关键能力？' -> 检索到 5 个文档
   查询2: '机械臂是什么？它的主要功能是什么？' -> 检索到 5 个文档
   查询3: '机械臂有哪些重要的特点和功能？' -> 检索到 5 个文档
   查询4: '机械臂的运作原理是什么？它有哪些关键能力？' -> 检索到 5 个文档
✨ 融合后共 13 个唯一文档
🏆 选择Top-5 文档:
   1. [分数: 0.0640] 尽
管，
机
械
臂的
运
动
速
度
可
以设定为
匀
速
运
动， 避
免
加
速
度变
化
而
造
成
对目
标
和
夹
爪之间动
力
学
平
衡
关系的影响
。
但
是
， 机械臂的
   2. [分数: 0.0489] 功能，其作用力的范围为 50N，精确度为 2.5N，准确度为 4N，扭矩范围为 10N·m，
精确度为 0.04N·m，准确度为 0.3N。机械臂本体自重 20.6kg，采用 220V交流供电，
   3. [分数: 0.0318] 机械臂 
可重复性  +-0.03mm  
有效负载  5 千克/11 磅 
工作半径  850mm/33.5 英寸 
自由度 6 个旋转关节
   4. [分数: 0.0315] ±180°/s，机械臂末端速度视各关节载荷与实际速度而定。机械臂本体的位姿可重复性
精度为 ±0.03mm。与其它型号机械臂不同的是，UR5e机械臂的工具法兰带有力感应
   5. [分数: 0.0161] 机记录该浮雕风格的接触表面图像，并使用光度立体算法
[55] 重建接触表面的深度图。
该传感器通过凝胶(gel)形变图像获取触觉信息，因此被称为Gelsight。为机器人手指


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.35s/it]


生成的查询:
  - 什么是机械臂？它有哪些关键能力？
  - 机械臂是什么？它的主要功能是什么？
  - 机械臂有哪些重要的特点和功能？
  - 机械臂的运作原理是什么？它有哪些关键能力？

Top-5 文档分数:
  1. 0.0640
  2. 0.0489
  3. 0.0318
  4. 0.0315
  5. 0.0161

答案:
机械臂是一种能够按照预定程序自动完成特定任务的机器人手臂。它通常由多个关节和执行器组成，可以实现精确的运动控制和抓取操作。机械臂的关键能力包括：
             1. 运动控制：机械臂可以实现精确的运动控制，包括速度、位置和方向控制。
             2. 抓取操作：机械臂可以实现精确的抓取操作，包括抓取物体和释放物体。
             3. 自动化操作：机械臂可以实现自动化操作，包括自动装配、搬运和拆卸等。
             4. 精确度：机械臂可以实现高精度的操作，包括精确的定位和精确的抓取。
             5. 功能：机械臂可以实现多种功能，包括搬运、装配、拆卸、切割、焊接、喷涂等。


### RAG-Fusion vs Multi-Query

| 特性 | Multi-Query | RAG-Fusion | 说明 |
|:-----|:-----------:|:----------:|:-----|
| 查询生成 | ✅ | ✅ | 都支持生成多个查询变体 |
| 并行检索 | ✅ | ✅ | 都支持并行检索多个查询 |
| 智能排序 | ❌ | ✅ RRF算法 | RAG-Fusion使用倒数排序融合 |
| 结果质量 | 中等 | 高 | RRF提升检索质量 |
| 计算成本 | 低 | 中等 | RRF增加计算开销 |
| 适用场景 | 一般检索 | 高质量需求 | 根据需求选择 |

## Part 3: Query Decomposition - 查询分解

核心概念：对于复杂的多步骤问题，Query Decomposition将其分解为多个子问题，分别回答后再合成最终答案。

**两种分解策略：**

1. 递归分解（Answer Recursively）

```python
复杂问题: "比较GPT-3和GPT-4在多模态能力上的差异"
    ↓
子问题1: "GPT-3有哪些能力？"
    ↓ 检索 + 回答
答案1: "GPT-3主要是文本模型..."
    ↓
子问题2: "GPT-4有哪些新能力？" (基于答案1)
    ↓ 检索 + 回答
答案2: "GPT-4增加了图像理解..."
    ↓
综合答案: "GPT-3仅支持文本，而GPT-4..."
```

2. 并行分解（Answer Individually）

```python
复杂问题: "比较Python和JavaScript在Web开发中的优劣"
    ↓
子问题1: "Python在Web开发中的优势"
子问题2: "JavaScript在Web开发中的优势"
子问题3: "Python在Web开发中的劣势"
子问题4: "JavaScript在Web开发中的劣势"
    ↓
并行检索 + 回答
    ↓
综合所有答案
```

#### 先实现递归分解

In [8]:
from typing import List, Dict, Any
from vllm import SamplingParams

class RecursiveDecomposition:
    """使用ChatML格式的递归查询分解"""
    
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever()
        self.llm = llm
        print("✅ 递归分解系统初始化完成")
        
    def decompose_query(self, question: str) -> List[str]:
        """使用ChatML格式分解复杂查询为子问题"""
        prompt = f"""<|im_start|>system
            你是一个AI助手，擅长将复杂问题分解为简单的子问题。
            
            请将以下复杂问题分解为2-4个简单的子问题：
            
            要求：
            1. 子问题应该按逻辑顺序排列
            2. 每个子问题都应该是独立可回答的
            3. 每行一个问题，不要编号<|im_end|>
            <|im_start|>user
            复杂问题：{question}
            
            请分解为子问题，每行一个：<|im_end|>
            <|im_start|>assistant
            """
        
        sampling_params = SamplingParams(
            max_tokens=512,
            temperature=0.5,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        response = outputs[0].outputs[0].text.strip()
        
        # 解析子问题
        sub_questions = []
        lines = response.split('\n')
        for line in lines:
            line = line.strip()
            if line and len(line) > 5:  # 过滤太短的子问题
                # 移除可能的编号标记
                if line[0].isdigit() and (line[1] == '.' or line[1] == '、'):
                    question = line[2:].strip()
                else:
                    question = line
                sub_questions.append(question)
        
        # 确保有子问题，如果没有则使用备选方案
        if not sub_questions:
            sub_questions = self._fallback_decomposition(question)
        
        return sub_questions[:4]  # 最多返回4个子问题
    
    def _fallback_decomposition(self, question: str) -> List[str]:
        """备选分解方案"""
        # 简单的基于规则的分解
        sub_questions = []
        
        if "比较" in question and "和" in question:
            # 比较类问题
            parts = question.split("比较")[1].split("和")
            if len(parts) >= 2:
                sub_questions.append(f"{parts[0].strip()}的主要特点是什么？")
                sub_questions.append(f"{parts[1].strip()}的主要特点是什么？")
                sub_questions.append("它们的主要差异是什么？")
        
        elif "步骤" in question or "流程" in question:
            # 步骤类问题
            sub_questions.append("第一步是什么？")
            sub_questions.append("关键步骤有哪些？")
            sub_questions.append("最终目标是什么？")
        
        else:
            # 通用分解
            sub_questions.append("基本概念是什么？")
            sub_questions.append("主要应用场景有哪些？")
            sub_questions.append("优势和局限性是什么？")
        
        return sub_questions
    
    def answer_sub_question(self, question: str, context: str = "") -> str:
        """使用ChatML格式回答单个子问题"""
        # 检索相关文档
        docs = self.retriever.get_relevant_documents(question)
        doc_context = "\n\n".join([doc.page_content for doc in docs[:3]])
        
        # 构建提示词
        if context:
            full_context = f"已知信息:\n{context}\n\n相关文档:\n{doc_context}"
        else:
            full_context = f"相关文档:\n{doc_context}"
        
        prompt = f"""<|im_start|>system
            你是一个专业的AI助手。请基于以下信息简洁地回答问题：
            
            {full_context}
            
            请遵循：
            1. 只使用提供的信息
            2. 保持回答简洁准确
            3. 如果信息不足，请说明<|im_end|>
            <|im_start|>user
            问题：{question}<|im_end|>
            <|im_start|>assistant
            """
        
        sampling_params = SamplingParams(
            max_tokens=300,
            temperature=0.3,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        return outputs[0].outputs[0].text.strip()
    
    def query(self, question: str) -> Dict[str, Any]:
        """执行递归分解查询"""
        print(f"🎯 原始问题: {question}")
        print("-" * 50)
        
        # 1. 分解问题
        sub_questions = self.decompose_query(question)
        print(f"🔍 分解为 {len(sub_questions)} 个子问题:")
        for i, sq in enumerate(sub_questions, 1):
            print(f"   {i}. {sq}")
        
        # 2. 递归回答
        accumulated_context = ""
        sub_answers = []
        
        print("\n💡 逐步回答子问题:")
        for i, sq in enumerate(sub_questions, 1):
            print(f"\n   子问题{i}: {sq}")
            answer = self.answer_sub_question(sq, accumulated_context)
            sub_answers.append(answer)
            accumulated_context += f"\n\n问题{i}: {sq}\n答案: {answer}"
            print(f"   答案: {answer[:100]}...")
        
        # 3. 综合最终答案
        final_prompt = f"""<|im_start|>system
            你是一个AI助手，需要基于子问题的答案来综合回答原始问题。
            
            原始问题：{question}
            
            子问题和答案：
            {self._format_sub_qa(sub_questions, sub_answers)}
            
            请基于以上子问题的答案，给出一个完整、连贯的综合答案。<|im_end|>
            <|im_start|>user
            请综合回答原始问题：{question}<|im_end|>
            <|im_start|>assistant
            """
        
        sampling_params = SamplingParams(
            max_tokens=500,
            temperature=0.3,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([final_prompt], sampling_params)
        final_answer = outputs[0].outputs[0].text.strip()
        
        return {
            "question": question,
            "sub_questions": sub_questions,
            "sub_answers": sub_answers,
            "final_answer": final_answer,
            "num_sub_questions": len(sub_questions)
        }
    
    def _format_sub_qa(self, questions: List[str], answers: List[str]) -> str:
        """格式化子问题和答案"""
        formatted = ""
        for i, (q, a) in enumerate(zip(questions, answers), 1):
            formatted += f"子问题{i}: {q}\n答案{i}: {a}\n\n"
        return formatted.strip()
    
    def batch_decompose(self, questions: List[str]) -> List[Dict]:
        """批量分解查询"""
        results = []
        for i, question in enumerate(questions, 1):
            print(f"\n🔍 处理第 {i}/{len(questions)} 个问题...")
            result = self.query(question)
            results.append(result)
        return results

In [11]:
print("🚀 ChatML格式的递归分解系统")
print("=" * 60)

# 1. 创建实例
decomp = RecursiveDecomposition(vectorstore, llm)

# 2. 执行查询
result = decomp.query("比较抓取检测和滑动检测的主要差异")

# 3. 查看结果
print("\n" + "="*60)
print("📋 子问题分解:")
for i, (q, a) in enumerate(zip(result["sub_questions"], result["sub_answers"]), 1):
    print(f"{i}. 问题: {q}")
    print(f"   答案: {a[:100]}...")
    print()

print("💡 最终综合答案:")
print(result["final_answer"])

🚀 ChatML格式的递归分解系统
✅ 递归分解系统初始化完成
🎯 原始问题: 比较抓取检测和滑动检测的主要差异
--------------------------------------------------


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.29s/it]


🔍 分解为 4 个子问题:
   1. 抓取检测和滑动检测都是用于检测物体在图像中的位置的方法，它们之间有什么主要差异？
   2. 抓取检测主要依赖于物体的形状和颜色，而滑动检测主要依赖于物体的运动特征，这两者之间有什么不同？
   3. 抓取检测和滑动检测在检测精度和速度方面有什么差异？
   4. 抓取检测和滑动检测在应用场景上有什么不同？

💡 逐步回答子问题:

   子问题1: 抓取检测和滑动检测都是用于检测物体在图像中的位置的方法，它们之间有什么主要差异？


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.26s/it]


   答案: 抓取检测和滑动检测都是用于检测物体在图像中的位置的方法，但它们之间存在一些主要差异。抓取检测主要关注物体在图像中的位置变化，而滑动检测则更关注物体在图像中的运动状态。抓取检测通常使用视觉信息，而滑动检...

   子问题2: 抓取检测主要依赖于物体的形状和颜色，而滑动检测主要依赖于物体的运动特征，这两者之间有什么不同？


...

   答案: 抓取检测和滑动检测在检测精度和速度方面存在差异。抓取检测主要关注物体在图像中的位置变化，而滑动检测则更关注物体在图像中的运动状态。抓取检测通常使用视觉信息，而滑动检测则可以使用触觉信息。此外，抓取检测...

   子问题4: 抓取检测和滑动检测在应用场景上有什么不同？


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.03it/s]


   答案: 抓取检测和滑动检测在应用场景上存在一些主要差异。抓取检测主要应用于物体定位和抓取，例如机器人抓取、无人机抓取等。而滑动检测则主要应用于物体运动状态的检测，例如物体滑动检测、物体滑动轨迹检测等。...


Processed prompts: 100%|██████████| 1/1 [00:02<00:00,  2.11s/it]


📋 子问题分解:
1. 问题: 抓取检测和滑动检测都是用于检测物体在图像中的位置的方法，它们之间有什么主要差异？
   答案: 抓取检测和滑动检测都是用于检测物体在图像中的位置的方法，但它们之间存在一些主要差异。抓取检测主要关注物体在图像中的位置变化，而滑动检测则更关注物体在图像中的运动状态。抓取检测通常使用视觉信息，而滑动检...

2. 问题: 抓取检测主要依赖于物体的形状和颜色，而滑动检测主要依赖于物体的运动特征，这两者之间有什么不同？
   答案: 抓取检测主要依赖于物体的形状和颜色，而滑动检测主要依赖于物体的运动特征。抓取检测通常使用视觉信息，而滑动检测则可以使用触觉信息。此外，抓取检测通常需要精确的物体定位，而滑动检测则更关注物体的运动轨迹。...

3. 问题: 抓取检测和滑动检测在检测精度和速度方面有什么差异？
   答案: 抓取检测和滑动检测在检测精度和速度方面存在差异。抓取检测主要关注物体在图像中的位置变化，而滑动检测则更关注物体在图像中的运动状态。抓取检测通常使用视觉信息，而滑动检测则可以使用触觉信息。此外，抓取检测...

4. 问题: 抓取检测和滑动检测在应用场景上有什么不同？
   答案: 抓取检测和滑动检测在应用场景上存在一些主要差异。抓取检测主要应用于物体定位和抓取，例如机器人抓取、无人机抓取等。而滑动检测则主要应用于物体运动状态的检测，例如物体滑动检测、物体滑动轨迹检测等。...

💡 最终综合答案:
抓取检测和滑动检测都是用于检测物体在图像中的位置的方法，但它们之间存在一些主要差异。抓取检测主要关注物体在图像中的位置变化，而滑动检测则更关注物体在图像中的运动状态。抓取检测通常使用视觉信息，而滑动检测则可以使用触觉信息。此外，抓取检测通常需要精确的物体定位，而滑动检测则更关注物体的运动轨迹。抓取检测和滑动检测在检测精度和速度方面存在差异。抓取检测主要应用于物体定位和抓取，例如机器人抓取、无人机抓取等。而滑动检测则主要应用于物体运动状态的检测，例如物体滑动检测、物体滑动轨迹检测等。


### 再实现并行分解

In [73]:
import asyncio
from typing import List, Dict, Any
from vllm import SamplingParams
from concurrent.futures import ThreadPoolExecutor
import time

class TrueAsyncParallelDecomposition:
    """真正的异步并行分解"""
    
    def __init__(self, vectorstore, llm, max_workers: int = 4):
        self.vectorstore = vectorstore
        self.llm = llm
        self.executor = ThreadPoolExecutor(max_workers=max_workers)
        print("✅ 真正异步并行分解系统初始化完成")
    
    async def _generate(self, prompt: str, max_tokens: int = 512) -> Dict[str, Any]:
        """异步生成"""
        sampling_params = SamplingParams(
            max_tokens=max_tokens,
            temperature=0.3,
            top_p=0.8
        )
        
        try:
            loop = asyncio.get_event_loop()
            outputs = await loop.run_in_executor(
                self.executor,
                lambda: self.llm.generate([prompt], sampling_params)
            )
            
            if outputs and outputs[0].outputs:
                text = outputs[0].outputs[0].text.strip()
                return {"success": True, "text": text}
            return {"success": False, "error": "生成失败"}
                
        except Exception as e:
            return {"success": False, "error": str(e)}
    
    async def answer_sub_question(self, question: str) -> Dict[str, Any]:
        """异步回答子问题"""
        start_time = time.time()
        
        try:
            # 1. 异步检索
            retriever = self.vectorstore.as_retriever()
            docs = await retriever.aget_relevant_documents(question)
            retrieval_time = time.time() - start_time
            
            if not docs:
                return {
                    "question": question,
                    "answer": "未找到相关文档",
                    "success": False,
                    "retrieval_time": retrieval_time
                }
            
            # 2. 构建上下文
            context = "\n".join([doc.page_content[:100] for doc in docs[:2]])
            
            # 3. 构建提示词
            prompt = f"""<|im_start|>system
                回答问题：<|im_end|>
                <|im_start|>user
                信息：{context}
                问题：{question}<|im_end|>
                <|im_start|>assistant
                """
            
            # 4. 异步生成
            gen_start = time.time()
            result = await self._generate(prompt, max_tokens=512)
            generation_time = time.time() - gen_start
            
            if result["success"]:
                return {
                    "question": question,
                    "answer": result["text"],
                    "success": True,
                    "retrieval_time": retrieval_time,
                    "generation_time": generation_time,
                    "total_time": time.time() - start_time
                }
            else:
                return {
                    "question": question,
                    "answer": f"生成失败: {result['error']}",
                    "success": False,
                    "retrieval_time": retrieval_time,
                    "generation_time": generation_time,
                    "total_time": time.time() - start_time
                }
            
        except Exception as e:
            return {
                "question": question,
                "answer": f"错误: {str(e)}",
                "success": False,
                "retrieval_time": 0,
                "generation_time": 0,
                "total_time": time.time() - start_time
            }
    
    async def query_parallel(self, question: str) -> Dict[str, Any]:
        """真正的异步并行查询"""
        print(f"🎯 问题: {question}")
        print("=" * 60)
        
        overall_start = time.time()
        
        # 1. 分解问题
        sub_questions = [
            "抓取检测是什么？",
            "滑动检测是什么？",
            "机械臂任务中抓取检测和滑动检测的关联是什么？"
        ]
        
        print(f"🔍 分解为 {len(sub_questions)} 个子问题:")
        for i, q in enumerate(sub_questions, 1):
            print(f"   {i}. {q}")
        
        # 2. 🔥 真正的并行执行（关键修复）
        print(f"⚡ 并行执行 {len(sub_questions)} 个子问题...")
        tasks = [self.answer_sub_question(q) for q in sub_questions]
        sub_results = await asyncio.gather(*tasks)  # 🔥 并行执行
        
        parallel_time = time.time() - overall_start
        
        # 3. 分析结果
        successful = [r for r in sub_results if r['success']]
        
        print(f"\n📊 执行统计:")
        print(f"   ✅ 成功: {len(successful)}/{len(sub_questions)}")
        print(f"   ⏱️  并行时间: {parallel_time:.2f}s")
        
        # 显示每个子问题的执行时间
        print("\n💡 子问题执行详情:")
        for i, result in enumerate(sub_results, 1):
            status = "✅" if result['success'] else "❌"
            print(f"   {i}. {status} 检索: {result['retrieval_time']:.2f}s, "
                  f"生成: {result['generation_time']:.2f}s, "
                  f"总计: {result['total_time']:.2f}s")
        
        # 4. 综合最终答案
        print("\n🧠 综合最终答案...")
        
        if successful:
            qa_context = "\n".join([
                f"Q{i}: {r['question']}\nA{i}: {r['answer']}" 
                for i, r in enumerate(successful, 1)
            ])
            
            synthesis_prompt = f"""<|im_start|>system
基于以下问答信息综合回答原始问题：<|im_end|>
<|im_start|>user
原始问题：{question}

子问题和答案：
{qa_context}

请给出综合答案：<|im_end|>
<|im_start|>assistant
"""
        else:
            synthesis_prompt = f"""<|im_start|>system
请回答以下问题：<|im_end|>
<|im_start|>user
问题：{question}<|im_end|>
<|im_start|>assistant
"""
        
        final_result = await self._generate(synthesis_prompt, max_tokens=512)
        total_time = time.time() - overall_start
        
        if final_result["success"]:
            final_answer = final_result["text"]
        else:
            final_answer = f"综合失败: {final_result['error']}"
        
        print(f"⏱️  总耗时: {total_time:.2f}s")
        print(f"📊 并行效率: {parallel_time/total_time*100:.1f}%")
        
        return {
            "question": question,
            "final_answer": final_answer,
            "sub_questions": sub_questions,
            "sub_results": sub_results,
            "metrics": {
                "total_time": total_time,
                "parallel_time": parallel_time,
                "success_rate": f"{len(successful)}/{len(sub_questions)}",
                "parallel_efficiency": f"{parallel_time/total_time*100:.1f}%"
            }
        }
    
    async def close(self):
        """关闭资源"""
        self.executor.shutdown()
        print("✅ 资源已关闭")


# === 性能对比测试 ===
async def performance_comparison():
    """性能对比：并行 vs 串行"""
    print("📊 性能对比测试")
    print("=" * 60)
    
    decomp = TrueAsyncParallelDecomposition(vectorstore, llm)
    
    try:
        question = "抓取检测和滑动检测在机械臂任务中的关联"
        
        # 并行执行
        print("🚀 并行执行测试...")
        parallel_start = time.time()
        parallel_result = await decomp.query_parallel(question)
        parallel_time = time.time() - parallel_start
        
        # 串行执行（模拟）
        print("\n🐌 串行执行测试...")
        serial_start = time.time()
        
        sub_questions = [
            "抓取检测是什么？",
            "滑动检测是什么？", 
            "机械臂任务中抓取检测和滑动检测的关联是什么？"
        ]
        
        serial_results = []
        for q in sub_questions:
            result = await decomp.answer_sub_question(q)  # 顺序执行
            serial_results.append(result)
        
        serial_time = time.time() - serial_start
        
        print(f"\n📈 性能对比结果:")
        print(f"   🚀 并行时间: {parallel_time:.2f}s")
        print(f"   🐌 串行时间: {serial_time:.2f}s")
        print(f"   ⚡ 加速比: {serial_time/parallel_time:.2f}x")
        print(f"   📊 效率提升: {(serial_time-parallel_time)/serial_time*100:.1f}%")
        
        return {
            "parallel_time": parallel_time,
            "serial_time": serial_time,
            "speedup": serial_time / parallel_time
        }
        
    finally:
        await decomp.close()


# === 使用示例 ===
async def true_async_demo():
    """真正异步并行演示"""
    print("🚀 真正异步并行分解演示")
    print("=" * 60)
    
    decomp = TrueAsyncParallelDecomposition(vectorstore, llm)
    
    try:
        result = await decomp.query_parallel("抓取检测和滑动检测与机械臂任务有什么关联？")
        
        print("\n" + "="*60)
        print("📋 最终结果:")
        print(f"问题: {result['question']}")
        print(f"成功率: {result['metrics']['success_rate']}")
        print(f"总耗时: {result['metrics']['total_time']:.2f}s")
        print(f"并行时间: {result['metrics']['parallel_time']:.2f}s")
        print(f"并行效率: {result['metrics']['parallel_efficiency']}")
        print(f"答案: {result['final_answer']}")
        
        return result
        
    finally:
        await decomp.close()


# 运行真正异步并行版本：
# result = await true_async_demo()

# 性能对比测试：
comparison = await performance_comparison()

📊 性能对比测试
✅ 真正异步并行分解系统初始化完成
🚀 并行执行测试...
🎯 问题: 抓取检测和滑动检测在机械臂任务中的关联
🔍 分解为 3 个子问题:
   1. 抓取检测是什么？
   2. 滑动检测是什么？
   3. 机械臂任务中抓取检测和滑动检测的关联是什么？
⚡ 并行执行 3 个子问题...


Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s]



Processed prompts:   0%|          | 0/2 [00:00<?, ?it/s]



📊 执行统计:
   ✅ 成功: 0/3
   ⏱️  并行时间: 0.20s

💡 子问题执行详情:
   1. ❌ 检索: 0.05s, 生成: 0.01s, 总计: 0.06s
   2. ❌ 检索: 0.05s, 生成: 0.13s, 总计: 0.17s
   3. ❌ 检索: 0.05s, 生成: 0.15s, 总计: 0.20s

🧠 综合最终答案...


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.08s/it]


⏱️  总耗时: 1.29s
📊 并行效率: 15.7%

🐌 串行执行测试...


Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.50s/it]


📈 性能对比结果:
   🚀 并行时间: 1.29s
   🐌 串行时间: 2.76s
   ⚡ 加速比: 2.14x
   📊 效率提升: 53.3%
✅ 资源已关闭


**分块策略对比**
| 特性 | 递归分解 | 并行分解 |
|:-----|:-------:|:-------:|
| 执行方式 | 顺序 | 并行 |
| 速度 | 慢 | 快 ⚡ |
| 子问题依赖 | 支持 | 不支持 |
| 适用场景 | 有逻辑顺序的问题 | 独立子问题 |
| 实现复杂度 | 中等 | 高 |

## Part 4: Step Back Prompting - 抽象化提问

核心概念：Step Back Prompting先提出一个更抽象、更概括的问题，获取背景知识后，再回答原始具体问题。

为什么需要Step Back？

```python
# ❌ 直接回答可能缺乏背景

原始问题: "Transformer中的Multi-Head Attention有几个头？"

直接检索 → 可能找不到确切答案（文档只描述了原理，没说具体数字）

# ✅ Step Back后

Step 1: Step Back问题
"Transformer架构的基本组成是什么？"

Step 2: 获取背景知识
"Transformer由编码器和解码器组成，使用Multi-Head Attention..."

Step 3: 结合背景回答原始问题
"根据原始论文，使用8个attention头..."
```

In [74]:
import asyncio
from typing import Dict, Any
from vllm import SamplingParams

class StepBackRAG:
    """Step Back提示的RAG系统"""
    
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.retriever = vectorstore.as_retriever()
        self.llm = llm
    
    def generate_step_back_question(self, question: str) -> str:
        """生成Step Back问题"""
        # ChatML格式提示词
        prompt = f"""<|im_start|>system
            给定一个具体问题，生成一个更抽象、更概括的问题。
            
            示例：
            具体问题: "GPT-4的上下文长度是多少？"
            Step Back问题: "GPT-4的主要技术特性有哪些？"
            
            具体问题: "Python中的装饰器如何工作？"
            Step Back问题: "Python中的元编程概念是什么？"<|im_end|>
            <|im_start|>user
            具体问题: {question}
            Step Back问题:<|im_end|>
            <|im_start|>assistant
            """
        
        sampling_params = SamplingParams(
            max_tokens=512,
            temperature=0.3,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        if outputs and outputs[0].outputs:
            step_back_q = outputs[0].outputs[0].text.strip()
        else:
            step_back_q = "生成失败"
        
        return step_back_q
    
    def query(self, question: str) -> dict:
        """执行Step Back RAG"""
        print(f"❓ 原始问题: {question}")
        
        # 1. 生成Step Back问题
        step_back_q = self.generate_step_back_question(question)
        print(f"📚 Step Back问题: {step_back_q}")
        
        # 2. 检索背景知识（Step Back问题）
        background_docs = self.retriever.get_relevant_documents(step_back_q)
        background = "\n\n".join([doc.page_content for doc in background_docs[:2]])
        
        # 3. 检索具体信息（原始问题）
        specific_docs = self.retriever.get_relevant_documents(question)
        specific = "\n\n".join([doc.page_content for doc in specific_docs[:2]])
        
        # 4. 结合两者生成答案
        # ChatML格式提示词
        prompt = f"""<|im_start|>system
            基于提供的背景知识和具体信息回答问题。
            先理解背景，再回答具体问题。<|im_end|>
            <|im_start|>user
            背景知识:
            {background}
            
            具体信息:
            {specific}
            
            原始问题: {question}
            请回答:<|im_end|>
            <|im_start|>assistant
            """
        
        sampling_params = SamplingParams(
            max_tokens=512,
            temperature=0.3,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        if outputs and outputs[0].outputs:
            answer = outputs[0].outputs[0].text.strip()
        else:
            answer = "生成失败"
        
        return {
            "question": question,
            "step_back_question": step_back_q,
            "answer": answer
        }



In [77]:
# 使用示例
step_back_rag = StepBackRAG(vectorstore, llm)
result = step_back_rag.query("滑动检测的操作流程？")

print("\n💡 答案:")
print(result["answer"])

❓ 原始问题: 滑动检测的操作流程？


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  7.72it/s]


📚 Step Back问题: 什么是滑动检测？


Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


💡 答案:
滑动检测的操作流程如下：

1. 滑动发生时，滑动检测算法会触发一个滑动信号。
2. 当目标发生滑动时，控制器会按照原始操作流程进行滑动检测。


Step Back的优势：

何时使用Step Back:
- 问题需要背景知识
- 直接检索效果不好
- 问题过于具体或技术性强
- 需要理论支撑答案

不适合的场景：
- 简单事实性问题
- 已有充足直接信息
- 实时数据查询


## Part 5: HyDE - 假设性文档嵌入

核心概念：HyDE (Hypothetical Document Embeddings) 不直接检索用户查询，而是先让LLM生成一个假设性的答案文档，然后用这个文档去检索相似内容。

**HyDE的直觉**

```python
# 传统检索
用户查询: "什么是机器学习？"
    ↓ 直接嵌入
查询向量: [0.1, 0.3, -0.2, ...]
    ↓ 检索
找到的文档

# HyDE检索
用户查询: "什么是机器学习？"
    ↓ LLM生成假设性答案
假设文档: "机器学习是人工智能的一个分支，它使计算机能够从数据中学习并做出预测..."
    ↓ 嵌入假设文档
文档向量: [0.2, 0.4, -0.1, ...]  # 与真实答案文档更相似！
    ↓ 检索
找到更相关的文档
```

为什么有效？

查询通常很短，而文档内容丰富。假设性文档比查询更接近真实文档的表达方式，因此能找到更相关的内容。


In [78]:
class HyDERAG:
    """HyDE (Hypothetical Document Embeddings) RAG"""
    
    def __init__(self, vectorstore, llm, embeddings):
        self.vectorstore = vectorstore
        self.llm = llm
        self.embeddings = embeddings
    
    def generate_hypothetical_document(
        self, 
        question: str,
        style: str = "academic"
    ) -> str:
        """生成假设性文档"""
        if style == "academic":
            system_msg = "你是一位专家。请对以下问题写一段详细、准确的学术性回答（200-300字）。"
        elif style == "concise":
            system_msg = "请对以下问题写一段简洁的回答（100-150字）。"
        else:
            system_msg = "请对以下问题写一段回答。"
        
        # ChatML格式提示词
        prompt = f"""<|im_start|>system
            {system_msg}<|im_end|>
            <|im_start|>user
            {question}<|im_end|>
            <|im_start|>assistant
            """
        
        from vllm import SamplingParams
        sampling_params = SamplingParams(
            max_tokens=400,
            temperature=0.3,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        if outputs and outputs[0].outputs:
            hypothetical_doc = outputs[0].outputs[0].text.strip()
        else:
            hypothetical_doc = "生成失败"
        
        return hypothetical_doc
    
    def search_with_hyde(
        self, 
        question: str, 
        k: int = 5
    ) -> List:
        """使用HyDE进行检索"""
        # 1. 生成假设性文档
        hyp_doc = self.generate_hypothetical_document(question)
        print(f"📝 假设性文档:\n{hyp_doc[:200]}...\n")
        
        # 2. 使用假设性文档检索
        # 注意：这里用假设文档而不是原始查询
        docs = self.vectorstore.similarity_search(hyp_doc, k=k)
        
        return docs
    
    def query(self, question: str, k: int = 5) -> dict:
        """执行HyDE RAG查询"""
        print(f"❓ 原始问题: {question}\n")
        
        # 1. HyDE检索
        docs = self.search_with_hyde(question, k=k)
        print(f"📚 检索到 {len(docs)} 个文档\n")
        
        # 2. 生成最终答案
        context = "\n\n".join([doc.page_content for doc in docs])
        
        # ChatML格式提示词
        prompt = f"""<|im_start|>system
            基于以下文档准确回答问题。<|im_end|>
            <|im_start|>user
            文档:
            {context}
            
            问题: {question}<|im_end|>
            <|im_start|>assistant
            """
        
        from vllm import SamplingParams
        sampling_params = SamplingParams(
            max_tokens=500,
            temperature=0.3,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        if outputs and outputs[0].outputs:
            answer = outputs[0].outputs[0].text.strip()
        else:
            answer = "生成失败"
        
        return {
            "question": question,
            "num_docs": len(docs),
            "answer": answer
        }

In [80]:
# 使用示例
hyde_rag = HyDERAG(vectorstore, llm, embeddings)
result = hyde_rag.query("解释滑动检测中的触觉传感器")

print("💡 最终答案:")
print(result["answer"])

❓ 原始问题: 解释滑动检测中的触觉传感器



Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.22s/it]


📝 假设性文档:
滑动检测中的触觉传感器是一种用于检测物体表面滑动的传感器。它们通常由一个弹性膜片和一个敏感元件组成，当物体滑过膜片时，敏感元件会感受到物体的滑动，并将其转化为电信号。这些传感器可以用于各种应用，如机器人导航、汽车安全系统和工业自动化。它们通常具有高精度、高灵敏度和快速响应时间的特点，可以有效地检测和识别物体的滑动行为。...

📚 检索到 5 个文档



Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  1.07it/s]

💡 最终答案:
触觉传感器是一种能够检测物体表面压力变化的传感器，当物体发生滑动时，触觉传感器会高频振动，触觉传感器数据会发生变化，从而引起压力中心数值的变化，通过分析压力中心数值的变化可以获得物体的滑动情况。滑动检测是机器人夹爪抓取控制的核心。


HyDE vs 传统检索

| 维度 | 传统检索 | HyDE |
|:-----|:-------:|:-----:|
| 检索对象 | 用户查询 | 假设性答案 |
| 语义匹配 | 查询↔文档 | 答案↔答案 |
| 查询长度敏感性 | 是 | 否 |
| 额外LLM调用 | 0 | 1 |
| 适用场景 | 清晰查询 | 复杂/技术性查询 |
| 检索质量 | 中等 | 高 |

# 技术对比与选择指南

**综合对比**

| 技术 | 召回率 | 准确率 | 速度 | 成本 | 复杂度 |
|:-----|:------:|:------:|:----:|:----:|:------:|
| Multi-Query | ⭐⭐⭐ | ⭐⭐ | ⭐⭐ | 💰💰 | ⭐ |
| RAG-Fusion | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐ | 💰💰💰 | ⭐⭐ |
| Decomposition | ⭐⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐ | 💰💰💰💰 | ⭐⭐⭐ |
| Step Back | ⭐⭐⭐ | ⭐⭐⭐⭐ | ⭐⭐ | 💰💰 | ⭐⭐ |
| HyDE | ⭐⭐⭐⭐ | ⭐⭐⭐⭐⭐ | ⭐⭐ | 💰💰 | ⭐⭐ |

**选择决策树**

```python
问题类型？
├─ 简单查询（单一事实）
│  → 不需要优化，使用基础RAG
│
├─ 表达模糊/多角度
│  → Multi-Query 或 RAG-Fusion
│
├─ 复杂多步骤问题
│  → Query Decomposition
│
├─ 需要背景知识
│  → Step Back
│
└─ 技术性强/专业领域
   → HyDE
```

**组合使用**

这些技术可以组合使用以获得更好的效果：


In [81]:
class AdvancedRAG:
    """组合多种技术的高级RAG系统"""
    
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.llm = llm
        
    def intelligent_query(self, question: str) -> dict:
        """智能选择和组合技术"""
        
        # 1. 分析查询复杂度
        complexity = self.analyze_complexity(question)
        
        if complexity == "simple":
            # 简单查询：直接检索
            return self.simple_rag(question)
        
        elif complexity == "ambiguous":
            # 模糊查询：Multi-Query + RRF
            return self.multi_query_with_fusion(question)
        
        elif complexity == "complex":
            # 复杂查询：分解 + HyDE
            return self.decomposition_with_hyde(question)
        
        elif complexity == "technical":
            # 技术查询：Step Back + HyDE
            return self.step_back_with_hyde(question)
    
    def analyze_complexity(self, question: str) -> str:
        """分析查询复杂度"""
        # ChatML格式提示词
        prompt = f"""<|im_start|>system
            分析查询的复杂度，返回以下之一:
            - simple: 简单事实性问题
            - ambiguous: 表达模糊的问题
            - complex: 需要多步推理的复杂问题
            - technical: 技术性强的专业问题
            
            只返回类别，不要解释。<|im_end|>
            <|im_start|>user
            {question}<|im_end|>
            <|im_start|>assistant
            """
        
        from vllm import SamplingParams
        sampling_params = SamplingParams(
            max_tokens=50,
            temperature=0.1,
            top_p=0.8,
            stop=["<|im_end|>", "<|endoftext|>"]
        )
        
        outputs = self.llm.generate([prompt], sampling_params)
        if outputs and outputs[0].outputs:
            complexity = outputs[0].outputs[0].text.strip().lower()
        else:
            complexity = "simple"  # 默认简单
        
        return complexity

# 性能优化与最佳实践

## 缓存策略

技术流程：

```python
用户输入问题 question
        ↓
调用 cached_multi_query(question)
        ↓
1. 生成缓存键：method + question → MD5 哈希值
        ↓
2. 检查缓存（query_cache）中是否存在该键？
        ├── 是 → 直接返回缓存中的查询列表（命中缓存）
        └── 否 → 
                ↓
            调用 generate_queries(question) 生成新查询
                ↓
            将结果存入缓存：query_cache[cache_key] = queries
                ↓
            返回新生成的查询列表
```

In [82]:
from functools import lru_cache
import hashlib

class CachedQueryOptimizer:
    """带缓存的查询优化器"""
    
    def __init__(self):
        self.query_cache = {}
        self.doc_cache = {}
    
    def get_cache_key(self, query: str, method: str) -> str:
        """生成缓存键"""
        content = f"{method}:{query}"
        return hashlib.md5(content.encode()).hexdigest()
    
    def cached_multi_query(self, question: str) -> List[str]:
        """缓存Multi-Query结果"""
        cache_key = self.get_cache_key(question, "multi_query")
        
        if cache_key in self.query_cache:
            print("💾 使用缓存的查询变体")
            return self.query_cache[cache_key]
        
        # 生成查询
        queries = generate_queries(question)
        self.query_cache[cache_key] = queries
        
        return queries

## 并行处理

```python
开始并行多查询处理
        ↓
输入参数：queries列表，retriever检索器，max_workers=5
        ↓
1. 获取当前事件循环：asyncio.get_event_loop()
        ↓
2. 创建线程池执行器：ThreadPoolExecutor(max_workers=max_workers)
        ↓
3. 遍历查询列表，为每个查询创建异步任务：
        ├── 查询1 → loop.run_in_executor(executor, retriever.get_relevant_documents, query1)
        ├── 查询2 → loop.run_in_executor(executor, retriever.get_relevant_documents, query2)
        ├── 查询3 → loop.run_in_executor(executor, retriever.get_relevant_documents, query3)
        ├── 查询4 → loop.run_in_executor(executor, retriever.get_relevant_documents, query4)
        └── 查询5 → loop.run_in_executor(executor, retriever.get_relevant_documents, query5)
        ↓
4. 使用asyncio.gather并行等待所有任务完成
        ↓
5. 收集每个查询的检索结果
        ↓
返回格式：List[List[Document]]
        ├── 结果1: query1的文档列表
        ├── 结果2: query2的文档列表
        ├── 结果3: query3的文档列表
        ├── 结果4: query4的文档列表
        └── 结果5: query5的文档列表  

In [84]:
import asyncio
from concurrent.futures import ThreadPoolExecutor

async def parallel_multi_query(
    queries: List[str],
    retriever,
    max_workers: int = 5
) -> List[List]:
    """并行执行多个查询"""
    loop = asyncio.get_event_loop()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        tasks = [
            loop.run_in_executor(
                executor,
                retriever.get_relevant_documents,
                query
            )
            for query in queries
        ]
        
        results = await asyncio.gather(*tasks)
    
    return results

## 自适应参数调整

In [86]:
def adaptive_k_selection(question: str, base_k: int = 5) -> int:
    """根据查询动态调整k值"""
    # 计算查询长度
    word_count = len(question.split())
    
    if word_count < 5:
        # 短查询需要更多文档
        return base_k + 2
    elif word_count > 20:
        # 长查询可以减少文档数
        return max(base_k - 1, 3)
    
    return base_k

# 使用
k = adaptive_k_selection("什么是AI？")  # 返回7
k = adaptive_k_selection("详细解释深度学习中的...")  # 返回4

# 案例1：构建智能客服助手

这是一个智能客服RAG系统，它能够根据用户问题的类型自动选择最合适的检索增强生成技术。

系统首先通过关键词匹配对用户查询进行分类，识别出四种主要场景：

• 产品信息类（包含"产品"、"功能"等关键词）：使用HyDE技术生成假设性答案进行检索，适合技术性强的专业问题

• 故障排查类（包含"问题"、"错误"等关键词）：采用递归分解技术，将复杂问题拆解为步骤化解决方案

• 操作教程类（包含"如何"、"怎么"等关键词）：运用Step Back技术，先理解背景知识再提供具体步骤

• 一般性问题：使用RAG-Fusion融合多种查询结果，确保回答的全面性

这种智能路由机制确保了不同类型的问题都能获得最合适的处理方式，大大提升了客服系统的准确性和用户体验。

In [87]:
class CustomerServiceRAG:
    """智能客服RAG系统"""
    
    def __init__(self, vectorstore, llm):
        self.vectorstore = vectorstore
        self.llm = llm
        
        # 预定义场景
        self.scenarios = {
            "product_info": ["产品", "功能", "特性", "价格"],
            "troubleshooting": ["问题", "错误", "不工作", "故障"],
            "how_to": ["如何", "怎么", "步骤", "教程"]
        }
    
    def classify_query(self, question: str) -> str:
        """分类用户查询"""
        for scenario, keywords in self.scenarios.items():
            if any(kw in question for kw in keywords):
                return scenario
        return "general"
    
    def handle_query(self, question: str) -> str:
        """处理客服查询"""
        scenario = self.classify_query(question)
        
        if scenario == "product_info":
            # 产品信息：使用HyDE获得更专业的答案
            return HyDERAG(self.vectorstore, self.llm).query(question)
        
        elif scenario == "troubleshooting":
            # 故障排查：使用分解获得步骤化答案
            return RecursiveDecomposition(self.vectorstore, self.llm).query(question)
        
        elif scenario == "how_to":
            # 教程类：使用Step Back获得背景+步骤
            return StepBackRAG(self.vectorstore, self.llm).query(question)
        
        else:
            # 一般查询：使用RAG-Fusion
            return RAGFusion(self.vectorstore, self.llm).query(question)

# 案例2：学术论文助手

这是一个学术论文助手RAG系统，专门为学术研究场景设计，集成了多种先进的检索增强生成技术来支持学术工作。

系统提供了两大核心功能：

• 文献综述生成：通过Multi-Query技术广泛检索相关研究现状，结合HyDE技术深入查找理论基础和核心概念，自动生成全面的文献综述

• 方法对比分析：利用递归分解技术将复杂的比较问题拆解为结构化子问题，系统性地分析不同方法的优缺点和适用场景

这个系统特别适合研究人员快速了解领域现状、梳理理论框架，以及进行深入的学术方法比较，大大提升了学术研究的效率和质量。

In [88]:
class AcademicRAG:
    """学术论文助手"""
    
    def __init__(self, vectorstore, llm):
        self.multi_query = MultiQueryRAG(vectorstore, llm)
        self.hyde = HyDERAG(vectorstore, llm, embeddings)
        self.decomp = RecursiveDecomposition(vectorstore, llm)
    
    def literature_review(self, topic: str) -> str:
        """文献综述"""
        # 1. Multi-Query找全面的相关论文
        broad_results = self.multi_query.query(
            f"关于{topic}的研究现状和主要发现"
        )
        
        # 2. HyDE找高质量的理论论文
        theory_results = self.hyde.query(
            f"{topic}的理论基础和核心概念"
        )
        
        # 3. 综合生成文献综述
        # ...
        
        return review
    
    def compare_methods(self, method1: str, method2: str) -> str:
        """方法对比"""
        # 使用查询分解
        comparison_query = f"比较{method1}和{method2}的优缺点"
        return self.decomp.query(comparison_query)